In [1]:
#### DO NOT RUN THIS CODE IN THE NOTEBOOK - copy paste into terminal in the HOME directory!

# git clone https://github.com/tensorflow/models.git

In [2]:
# !pip install s3fs -q
# !pip install tensorflow_hub -q
# !pip install tensorflow_datasets -q
# !pip install pycocotools -q
# !pip install gin-config -q
# !pip install immutabledict -q
# !pip install sentencepiece -q
# !pip install transformers -q
# !pip install -r official/projects/movinet/requirements.txt -q

In [2]:
import configparser

config = configparser.ConfigParser()
config.read('/home/ec2-user/.aws/credentials')

['/home/ec2-user/.aws/credentials']

In [3]:
# AWS credentials and S3 settings
aws_access_key_id = config["default"]['aws_access_key_id']
aws_secret_access_key = config["default"]['aws_secret_access_key']
bucket_name = 'asl-capstone'
s3_URI = 's3://asl-capstone/'

In [4]:
import s3fs
fs = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)

import boto3
s3 = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key,
                  region_name = 'us-west-2')


In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

tf.keras.backend.clear_session()

2023-11-08 04:33:53.571478: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-08 04:33:55.601836: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
%cd /home/ec2-user/models

/home/ec2-user/models


In [7]:
pwd

'/home/ec2-user/models'

In [9]:
model_version = 'a3'

if model_version=='a3':
  max_frames = 120
  image_dims = (256,256)
elif model_version=='a0':
  max_frames = 50
  image_dims = (172,172)
elif model_version=='a5':
  max_frames = 120
  image_dims = (320,302)

In [10]:
# Get the kinetics-600 action labels
KINETICS_URL = "official/projects/movinet/files/kinetics_600_labels.txt"
with open(KINETICS_URL) as obj:
  labels_600 = [line for line in obj.readlines()]
print("Found %d labels." % len(labels_600))

Found 600 labels.


In [11]:
# # Get the kinetics-400 action labels from the GitHub repository.
# from urllib import request  # requires python3
# KINETICS_URL = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt"
# with request.urlopen(KINETICS_URL) as obj:
#   labels_400 = [line.decode("utf-8").strip() for line in obj.readlines()]
# print("Found %d labels." % len(labels_400))

In [12]:
# !export XLA_PYTHON_CLIENT_PREALLOCATE=false
# !export XLA_FLAGS="--xla_gpu_strict_conv_algorithm_picker=false --xla_gpu_force_compilation_parallelism=1"

In [13]:
# from official.projects.movinet.modeling import movinet_model

# model = movinet_model.MovinetClassifier(
#     backbone, num_classes=600, output_states=False) #change numclasses for dataset

# # Create your example input here.
# # Refer to the paper for recommended input shapes.
# inputs = tf.ones([1, 120, image_dims[0], image_dims[1], 3]) #make sure input shape corresponds to correct model

# # Build the model
# model.build(inputs.shape)

# # load pretrained checkpoint
# checkpoint_dir = f'movinet_{model_version}_base'
# checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
# checkpoint = tf.train.Checkpoint(model=model)
# status = checkpoint.restore(checkpoint_path)
# status.assert_existing_objects_matched()

# # Run the model prediction.
# output = model(videos)
# prediction = tf.argmax(output, -1)
    
# print(labels_600[prediction.numpy()[0]])
# print(labels_600[prediction.numpy()[1]])

In [8]:
video_files = pd.read_parquet("s3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/masterfile.parquet")
video_files = video_files.drop_duplicates().reset_index(drop=True)
print(video_files.shape)
video_files.head(10)

(484, 2)


,caption,path
0,Welcome to the third round,s3://asl-capstone/youtube-asl/test_sample/nump...
1,of the Pearls announcement.,s3://asl-capstone/youtube-asl/test_sample/nump...
2,Today's category is called the,s3://asl-capstone/youtube-asl/test_sample/nump...
3,Hidden Pearls. What is it?,s3://asl-capstone/youtube-asl/test_sample/nump...
4,"Hidden Pearls...in other words,",s3://asl-capstone/youtube-asl/test_sample/nump...
5,they are known as the dark horses.,s3://asl-capstone/youtube-asl/test_sample/nump...
6,"Every now and then, they are seldom recognized",s3://asl-capstone/youtube-asl/test_sample/nump...
7,for their work or performance.,s3://asl-capstone/youtube-asl/test_sample/nump...
8,"Additionally, they are behind a big leader",s3://asl-capstone/youtube-asl/test_sample/nump...
9,"If not for them, the big leader wouldn't be here.",s3://asl-capstone/youtube-asl/test_sample/nump...


In [15]:
# def data_generator(file_paths_df, n_frames=120):
#     for _, row in file_paths_df.iterrows():
#         with fs.open(row['path'], 'rb') as f:
#             video = np.load(f)

#         video = pad_video_frames(video, n_frames=n_frames)
#         yield video, row['caption']


# # Create TensorFlow Dataset for model input. 
# train_dataset = tf.data.Dataset.from_generator(
#     generator=lambda: data_generator(video_files, n_frames=120),
#     output_signature=(
#         tf.TensorSpec(shape=(120, 256, 256, 3), dtype=tf.float32),
#         tf.TensorSpec(shape=(), dtype=tf.string)
#     )
# )

In [36]:
videos = []

# finished:
#100, 150, 200, 

for i in tqdm([250, 300], 350, 400, 450, 484):
    with fs.open(f"s3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/padded_joined_videos_{i}.npy", 
                 'rb') as f:
        video = np.load(f)

    videos.append(video)

videos = np.concatenate(videos)
videos.shape

 67%|████████████████████████████████████████████████████████████████████████████▋                                      | 2/3 [45:05<22:32, 1352.70s/it]


KeyboardInterrupt: 

In [38]:
videos = np.concatenate(videos)
videos.shape

(100, 120, 256, 256, 3)

In [40]:
captions = video_files.caption.to_list()[201:301]
print(videos.shape)
print(len(captions))
# np.save('video_1.npy', videos.numpy())
# np.save('video_2.npy', videos)

(100, 120, 256, 256, 3)
100


In [25]:
from official.projects.movinet.modeling import movinet

# Create backbone and model.
backbone = movinet.Movinet(
    model_id=model_version, #change to correspond to model
    causal=False,
    use_external_states=False,
)

2023-11-02 05:53:26.751699: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-02 05:53:27.136449: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-02 05:53:27.137335: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [41]:
%%time

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# tf.keras.backend.clear_session()

# movinet_output = backbone(videos)

## Takes 4 mins to run 10 videos of size (120, 256, 256, 3)

batch_size = 10
embeddings_chunks = []

# for i in range(100,400, 50):    
#     with fs.open(f"s3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/padded_joined_videos_{i}.npy", 
#                  'rb') as f:
#         videos = np.load(f)

for vid in chunks(videos, batch_size):
    # Generate video embeddings and store to CPU
    embeddings = backbone(vid)[0]['block4_layer3'].cpu().numpy()
    embeddings_chunks.append(embeddings)


movinet_output = np.concatenate(embeddings_chunks)
print(movinet_output.shape)

2023-11-02 06:56:41.167965: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.93GiB (rounded to 3149925120)requested by op __inference_call_200700
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-11-02 06:56:41.168066: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-11-02 06:56:41.168083: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 212, Chunks in use: 212. 53.0KiB allocated for chunks. 53.0KiB in use in bin. 17.7KiB client-requested in use in bin.
2023-11-02 06:56:41.168096: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 193, Chunks in use: 193. 122.8KiB allocated for chunks. 122.8KiB in use in bin. 97.5KiB client-requested in use in bin.
2023-11-02 06:56:41.168108: I t

ResourceExhaustedError: Exception encountered when calling layer 'conv3d' (type Conv3D).

Out of memory while trying to allocate 3149924992 bytes.
BufferAssignment OOM Debugging.
BufferAssignment stats:
             parameter allocation:    1.17GiB
              constant allocation:         0B
        maybe_live_out allocation:    2.93GiB
     preallocated temp allocation:    2.93GiB
  preallocated temp fragmentation:         0B (0.00%)
                 total allocation:    7.04GiB
              total fragmentation:    4.00MiB (0.06%)
Peak buffers:
	Buffer 1:
		Size: 2.93GiB
		Operator: op_type="Conv3D" op_name="Conv3D" source_file="/opt/tensorflow/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py" source_line=262
		XLA Label: custom-call
		Shape: f32[10,40,120,128,128]
		==========================

	Buffer 2:
		Size: 2.93GiB
		Operator: op_type="Conv3D" op_name="Conv3D" source_file="/opt/tensorflow/lib/python3.10/site-packages/keras/src/layers/convolutional/base

In [27]:
# with fs.open(f"s3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/movinet_embedding_300.npy", 
#              'wb') as f:
#     np.save(f, movinet_output)

In [29]:
# tf.keras.layers.GlobalAveragePooling3D(keepdims=True)(vid_embedding)
# tf.keras.layers.Flatten()(vid_embedding)

vid_embedding = tf.keras.layers.Reshape((movinet_output.shape[1], 8 * 8 * 168), name='flatten')(movinet_output)

vid_embedding = tf.keras.layers.Dense(1024, activation='linear', name='hidden_layer')(vid_embedding)

# vid_embedding_flatten = tf.keras.layers.Normalization(axis=2)(vid_embedding_flatten)

print(vid_embedding.shape)

(201, 120, 1024)


2023-11-02 06:04:16.015407: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [31]:
# with fs.open(f"s3://asl-capstone/youtube-asl/test_sample/numpy_files/RGB/movinet_embedding_flattened_350.npy", 
#              'wb') as f:
#     np.save(f, vid_embedding.numpy())

inputs = [batch, frames, height, width, channels]

vid_embeddings = model_backbone(inputs)[0]['block4_layer3']

hidden = tf.keras.layer(vid_embeddings)

dropout

transformer

decoder

In [15]:
# Load model directly
from transformers import AutoTokenizer, TFT5ForConditionalGeneration #, TFT5EncoderModel
# from transformers.modeling_tf_outputs import TFBaseModelOutputWithPastAndCrossAttentions

# with tf.device("/GPU:0"):
tokenizer = AutoTokenizer.from_pretrained("google/t5-v1_1-large")
lang_model = TFT5ForConditionalGeneration.from_pretrained("google/t5-v1_1-large")
    # encoder_model = T5EncoderModel.from_pretrained("google/t5-v1_1-large")

lang_model.trainable = True

# tokenizer = AutoTokenizer.from_pretrained("t5-large")
# model = TFT5ForConditionalGeneration.from_pretrained("t5-large")
# encoder_model = T5EncoderModel.from_pretrained("t5-large")



You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
2023-11-08 04:35:13.621334: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-08 04:35:14.073426: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUM

In [ ]:
# # training
# inputs = tokenizer("The <extra_id_0> walks in <extra_id_1> park", return_tensors="tf").input_ids
# labels = tokenizer("<extra_id_0> cute dog <extra_id_1> the <extra_id_2>", return_tensors="tf").input_ids
# outputs = model(inputs, labels=labels)
# loss = outputs.loss
# logits = outputs.logits
# print(loss)
# print(logits)

# # inference
# inputs = tokenizer(
#    [ "studies have shown that owning a cat is good for you", "i have always wanted to have a pet"], return_tensors="pt", padding=True
# ).input_ids  # Batch size 1

# # Convert from tokens to embeddings
# encoder_outputs = encoder_model(input_ids=inputs)
# encoder_outputs['last_hidden_state'] = encoder_outputs['last_hidden_state'].detach().numpy()
# print(encoder_outputs['last_hidden_state'].shape)

# # Generate text from embeddings
# outputs = model.generate(encoder_outputs=encoder_outputs)
# # outputs = model.generate(inputs)


# print("Response: \n")
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
# # studies have shown that owning a dog is good for you

In [13]:
print(vid_embedding.shape)
print(len(captions))

(201, 120, 1024)
201


In [19]:
# training
# input_mask = np.ones((2,100,1024))
encoder_outputs = tf.convert_to_tensor(vid_embedding)

labels = tokenizer(captions, return_tensors="tf", padding=True).input_ids
outputs = lang_model(input_ids=None, inputs_embeds=encoder_outputs, labels=labels, training=True)
print(outputs.loss)
# loss = outputs.loss
# logits = outputs.logits
# print(loss)
# print(logits)

tf.Tensor([54.503277], shape=(1,), dtype=float32)


In [23]:
captions[0]

'Welcome to the third round'

In [21]:
# model.generate(encoder_outputs=encoder_outputs)
tokenizer.decode(lang_model.generate(inputs_embeds=encoder_outputs)[0], skip_special_tokens=True)

/opt/tensorflow/lib/python3.10/site-packages/transformers/generation/tf_utils.py:838: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2023-11-08 04:39:47.301175: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55703850 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-08 04:39:47.301212: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A10G, Compute Capability 8.6
2023-11-08 04:39:47.408791: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-08 04:39:47.854060: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8903
2023-11-08 04:39:48.206243: I ./tensorflow/compiler/jit/device_compiler.h:186]

's its its it so it. so when it so it so it so it'

In [ ]:
def movinet_T5_model(video_base_model, language_base_model,
                      max_sequence_length=120,
                      hidden_size = 1024,
                      dropout=0.1,
                      learning_rate=0.00005):
    """
    Builds a translation model that accepts videos, creates video embeddings using a MoViNet model,
    and then passes the video embeddings into an encoder-decoder to generate text translation
    """

    # Set all models to trainable
    video_base_model.trainable = True
    language_base_model.trainable = True

    # Set input structure for video inputs
    video_inputs = tf.keras.layers.Input(shape=(max_sequence_length, 256, 256, 3), dtype=tf.int64, name='input_layer')
    # token_type_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='attention_mask_layer')

    # Get output embeddings from video model
    movinet_out = video_base_model(video_inputs)
    vid_embedding = movinet_out[0]['block4_layer3']

    # Layer to flatten w x h x c dimensions
    vid_embedding_flatten = tf.keras.layers.Reshape((120, 8 * 8 * 168), name='flatten')(vid_embedding)

    # Hidden layer to force embeddings into correct shape for language model
    vid_embedding_flatten = tf.keras.layers.Dense(1024, activation='linear', name='hidden_layer')(vid_embedding_flatten)

    # # Pass embeddings into T5 language model
    # language_output = language_base_model.generate(encoder_outputs=encoder_outputs, attention_mask=attention_mask)

    # classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)

    # classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

    # classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    #                              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    #                              metrics='accuracy')



    ### END YOUR CODE

    model = tf.keras.Model(inputs=inputs, outputs=vid_embedding_flatten)

    return model